# A. Separacion de conjuntos de prueba y entrenamiento

## 0. Extra: Agregar dataset de Kaggle

In [1]:
# Requisitos previos
! pip install kaggle
! mkdir ~/.kaggle
! cp drive/MyDrive/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Descargar dataset de competicion
! kaggle competitions download <name>

In [2]:
# Descargar dataset
! kaggle datasets download sanjeetsinghnaik/top-1000-highest-grossing-movies

  0% 0.00/106k [00:00<?, ?B/s]
100% 106k/106k [00:00<00:00, 60.8MB/s]


In [3]:
# Descomprimimos el zip
! unzip top-1000-highest-grossing-movies.zip

Archive:  top-1000-highest-grossing-movies.zip
  inflating: Highest Holywood Grossing Movies.csv  


## 1. Separar el dataset

Cargamos el dataset

In [4]:
import pandas as pd
dataset = pd.read_csv('Highest Holywood Grossing Movies.csv')

Observacion rapida

In [5]:
dataset.head()

,Unnamed: 0,Title,Movie Info,Distributor,Release Date,Domestic Sales (in $),International Sales (in $),World Sales (in $),Genre,Movie Runtime,License
0,0,Star Wars: Episode VII - The Force Awakens (2015),"As a new threat to the galaxy rises, Rey, a de...",Walt Disney Studios Motion Pictures,"December 16, 2015",936662225,1132859475,2069521700,"['Action', 'Adventure', 'Sci-Fi']",2 hr 18 min,PG-13
1,1,Avengers: Endgame (2019),After the devastating events of Avengers: Infi...,Walt Disney Studios Motion Pictures,"April 24, 2019",858373000,1939128328,2797501328,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13
2,2,Avatar (2009),A paraplegic Marine dispatched to the moon Pan...,Twentieth Century Fox,"December 16, 2009",760507625,2086738578,2847246203,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",2 hr 42 min,PG-13
3,3,Black Panther (2018),"T'Challa, heir to the hidden but advanced king...",Walt Disney Studios Motion Pictures,NaN,700426566,647171407,1347597973,"['Action', 'Adventure', 'Sci-Fi']",2 hr 14 min,NaN
4,4,Avengers: Infinity War (2018),The Avengers and their allies must be willing ...,Walt Disney Studios Motion Pictures,NaN,678815482,1369544272,2048359754,"['Action', 'Adventure', 'Sci-Fi']",2 hr 29 min,NaN


In [6]:
dataset.shape

(918, 11)

Separamos el dataset

In [7]:
import numpy as np
X = dataset.values
X = np.delete(X,7,axis=1)
Y = dataset.values[:,7]

In [8]:
print("X:",X.shape)
print("Y:",Y.shape)
print(Y[:5])

X: (918, 10)
Y: (918,)
[2069521700 2797501328 2847246203 1347597973 2048359754]


In [9]:
from sklearn.model_selection import train_test_split
X_train, Y_train, X_test, Y_test = train_test_split(X,Y, train_size=0.8, random_state=1)

In [10]:
# Data de entrenamiento
print("X_train:",X_train.shape)

X_train: (734, 10)


# B. Data categorica

Consultamos el numero de clases por variable

In [11]:
# Caracteristica: Licencia
dataset['License'].value_counts()

PG-13    363
R        194
PG       173
G         14
Name: License, dtype: int64

In [12]:
license = dataset['License'].sample(n=10, random_state=1)
license = license.values

## 1. Label Encoder

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
license_le = le.fit_transform(license)

In [14]:
print("License:",license)
print("T. License:",license_le)

License: ['R' 'PG-13' nan 'PG-13' 'PG' nan 'R' 'PG-13' 'PG' nan]
T. License: [2 1 3 1 0 3 2 1 0 3]


## 2. Label Binarizer

In [16]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
license_lb = lb.fit_transform(license_le)

In [17]:
print("T. License:",license_lb)

T. License: [[0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]]


## 3. DictVectorizer

In [24]:
data = [
        {'f1': 0.2, 'f2': -0.2, 'f3': 10, 'f4': 0.6},
        {'f1': 0.1, 'f2': 15, 'f3': 5, 'f4': 6},
        {'f1': 0.5, 'f2': 12, 'f3': 11, 'f4': 16},
        {'f1': 0, 'f2': 15, 'f3': 1, 'f4': 0.2}
]

In [25]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer()
data_dv = dv.fit_transform(data)

In [26]:
data_dv.todense()

matrix([[ 0.2, -0.2, 10. ,  0.6],
        [ 0.1, 15. ,  5. ,  6. ],
        [ 0.5, 12. , 11. , 16. ],
        [ 0. , 15. ,  1. ,  0.2]])

In [23]:
# Observar el diccionario de palabras
dv.vocabulary_

{'f1': 0, 'f2': 1, 'f3': 2, 'f4': 3}

In [29]:
dict_vect = data_dv.todense()
dict_vect[0,:]

matrix([[ 0.2, -0.2, 10. ,  0.6]])

In [30]:
license_le.shape

(10,)

In [32]:
data_vect = np.zeros((10,4))
for i in range(10):
  indice = license_le[i]
  data_vect[i,:] = dict_vect[indice,:]

In [33]:
data_vect

array([[ 0.5, 12. , 11. , 16. ],
       [ 0.1, 15. ,  5. ,  6. ],
       [ 0. , 15. ,  1. ,  0.2],
       [ 0.1, 15. ,  5. ,  6. ],
       [ 0.2, -0.2, 10. ,  0.6],
       [ 0. , 15. ,  1. ,  0.2],
       [ 0.5, 12. , 11. , 16. ],
       [ 0.1, 15. ,  5. ,  6. ],
       [ 0.2, -0.2, 10. ,  0.6],
       [ 0. , 15. ,  1. ,  0.2]])

## 4. FeatureHasher

In [37]:
texto = "Hola mundo"
texto.__hash__()

2246317922190804267

In [41]:
from sklearn.feature_extraction import FeatureHasher
fh = FeatureHasher()
data_fh = fh.fit_transform(data)

In [43]:
data_fh.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [44]:
data_fh.shape

(4, 1048576)

In [49]:
data_fh[data_fh>0]

matrix([[ 0.2, 10. , 15. ,  0.1,  5. , 12. ,  0.5, 11. , 15. ,  1. ]])

## 5. Encoder ordinario

In [51]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
license_oe = oe.fit_transform(license.reshape(-1,1))

In [52]:
print("License:",license)
print("T. License:",license_oe)

License: ['R' 'PG-13' nan 'PG-13' 'PG' nan 'R' 'PG-13' 'PG' nan]
T. License: [[ 2.]
 [ 1.]
 [nan]
 [ 1.]
 [ 0.]
 [nan]
 [ 2.]
 [ 1.]
 [ 0.]
 [nan]]


## 6. One Hot Encoder

In [53]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
license_ohe = ohe.fit_transform(license.reshape(-1,1))

In [55]:
print("License:",license)
print("T. License:")
print(license_ohe.todense())

License: ['R' 'PG-13' nan 'PG-13' 'PG' nan 'R' 'PG-13' 'PG' nan]
T. License:
[[0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]


# C. Manejo de data NaN

In [57]:
sub_dataset = dataset.sample(n=10, random_state=1)
sub_dataset = sub_dataset.values
sub_dataset = np.delete(sub_dataset, 2, axis=1)
sub_dataset

array([[900, 'Alien (1979)', 'Twentieth Century Fox', 'May 25, 1979',
        81900459, 24385063, 106285522, "['Horror', 'Sci-Fi']",
        '1 hr 57 min', 'R'],
       [570, 'Notting Hill (1999)', 'Universal Pictures', 'May 28, 1999',
        116089678, 247800000, 363889678,
        "['Comedy', 'Drama', 'Romance']", '2 hr 4 min', 'PG-13'],
       [791, 'Save the Last Dance (2001)', 'Paramount Pictures', nan,
        91057006, 40649803, 131706809, "['Drama', 'Music', 'Romance']",
        '1 hr 52 min', nan],
       [189, 'Venom (2018)', 'Sony Pictures Entertainment (SPE)',
        'October 3, 2018', 213515506, 642569645, 856085151,
        "['Action', 'Adventure', 'Sci-Fi']", '1 hr 52 min', 'PG-13'],
       [372, 'Puss in Boots (2011)', 'DreamWorks', 'October 27, 2011',
        149260504, 405726973, 554987477,
        "['Adventure', 'Animation', 'Comedy', 'Family', 'Fantasy']",
        '1 hr 30 min', 'PG'],
       [191, 'Venom: Let There Be Carnage (2021)',
        'Sony Pictures Enter

In [60]:
sub_dataset[3:6,5] = np.nan

In [62]:
#sub_dataset

### 1. Imputer

Data numerica

In [67]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="mean")

In [70]:
# Antes
sub_dataset[:,4:7]

array([[81900459, 24385063, 106285522],
       [116089678, 247800000, 363889678],
       [91057006, 40649803, 131706809],
       [213515506, nan, 856085151],
       [149260504, nan, 554987477],
       [212638437, nan, 501138437],
       [106580051, 231000000, 337580051],
       [129128133, 120220800, 249348933],
       [334191110, 691277106, 1025468216],
       [82571173, 47583395, 130154568]], dtype=object)

In [69]:
imputer.fit_transform(sub_dataset[:,4:7])

array([[8.19004590e+07, 2.43850630e+07, 1.06285522e+08],
       [1.16089678e+08, 2.47800000e+08, 3.63889678e+08],
       [9.10570060e+07, 4.06498030e+07, 1.31706809e+08],
       [2.13515506e+08, 2.00416595e+08, 8.56085151e+08],
       [1.49260504e+08, 2.00416595e+08, 5.54987477e+08],
       [2.12638437e+08, 2.00416595e+08, 5.01138437e+08],
       [1.06580051e+08, 2.31000000e+08, 3.37580051e+08],
       [1.29128133e+08, 1.20220800e+08, 2.49348933e+08],
       [3.34191110e+08, 6.91277106e+08, 1.02546822e+09],
       [8.25711730e+07, 4.75833950e+07, 1.30154568e+08]])

In [71]:
# Observamos las estadisticas
imputer.statistics_

array([1.51693206e+08, 2.00416595e+08, 4.25664484e+08])

Data categorica

In [72]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="most_frequent")

In [73]:
# Antes
sub_dataset[:,-1]

array(['R', 'PG-13', nan, 'PG-13', 'PG', nan, 'R', 'PG-13', 'PG', nan],
      dtype=object)

In [75]:
imputer.fit_transform(sub_dataset[:,-1].reshape((-1,1)))

array([['R'],
       ['PG-13'],
       ['PG-13'],
       ['PG-13'],
       ['PG'],
       ['PG-13'],
       ['R'],
       ['PG-13'],
       ['PG'],
       ['PG-13']], dtype=object)